# Source modelling
awojdyla@lbl.gov, October 2021

In this script, we model the source for an undulator

–adapted from `cosmicu_fdr.m`

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Conversions and units

In [2]:
# speed of light [m/s]
c   = 299792458         
# Planck constanct [J.s]
h   = 6.626070040*1e-34 
# elementaty charge x 1V [J.s]
eV  = 1.6021766208*1e-19
# Avogadro number [mol-1]
Ag  = 6.0221408571e23   
# degree/rad conversion
deg = 180/np.pi
# gaussian fwhm to variance
sigma_fhwm = 1/(2*np.sqrt(2*np.log(2)))
# inch to meter conversion
in_m = 25.4e-3          

In [3]:
# strictly equivalent

import scipy.constants as constant
#constant.c
#constant.h
#constant.e

from scipy.constants import c, h, e


In [4]:
# storage ring energy
Esr_GeV = 2.0
# Storage ring current
Isr_A = 500e-3
# energy spread
dEpE = 0.98e-3

## Undulator source

### Source size

In [5]:
# undulator
undulator_N = 55
undulator_L_m = 2.071
Lambda0_m = 38e-3

# operating range
Es_eV = np.linspace(250,2500,226)
lambdas_m = 1.2398e-06/Es_eV

# electron beam size and divergence
sex_m = 12.33e-6
sey_m = 15.10e-6
sexp_rad = 6.164e-6
seyp_rad = 5.033e-6

# photon beam size
sp_m    = np.sqrt(lambdas_m*2*undulator_L_m)/(2*np.pi)
spp_rad = np.sqrt(lambdas_m/(2*undulator_L_m))
# convoluted beam size
Sx_m  = np.sqrt(sex_m**2+sp_m**2)
Sxp_rad = np.sqrt(sexp_rad**2+spp_rad**2)
Sy_m  = np.sqrt(sey_m**2+sp_m**2)
Syp_rad = np.sqrt(seyp_rad**2+spp_rad**2)

#[Sx_m, Sxp_rad, Sy_m, Syp_rad] = Gen4.beam_size(Es_eV, undulator_L_m)

### tuning curves

In [6]:
# Deflection parameter
K = lambda n_harm, E_eV: np.sqrt(2*(9.5*n_harm*Esr_GeV**2./(Lambda0_m*E_eV)-1))

### Flux

In [7]:
from scipy import special

N_und = undulator_N
n_harm = 1
E_eV = 300

# See KJ Kim eq 42
Y = lambda n, K:  n*K**2/(4*(1+K**2/2))
F = lambda n, K: (n*K/(1+(K**2)/2))**2*(special.jv((n+1)/2, Y(n,K))-special.jv((n-1)/2, Y(n,K)))**2
Q = lambda n, K: (1+(K**2)/2)*F(n,K)/n


flux_ph_s_01bw = 0.72 * 1e14*N_und*Q(n_harm,K(n_harm,E_eV))*Isr_A
print(("Photon energy %1.1f eV " %E_eV)+("on harmonic %i" %n_harm) + (" at K = %1.3f" % K(n_harm,E_eV)))
print("Flux at resonance: %1.3e [ph/sec/0.1BW]"%flux_ph_s_01bw)

Photon energy 300.0 eV on harmonic 1 at K = 2.160
Flux at resonance: 1.762e+15 [ph/sec/0.1BW]


### Emittance
Warning: untested

In [8]:
from scipy import special

Qan_h = lambda xe: np.sqrt(2*xe**2./(np.exp(-2*xe**2)+np.sqrt(2*np.pi)*xe*special.erf(np.sqrt(2)*xe)-1))
Qsn_h = lambda xe: Qan_h(xe/4)**(2/3)
# adding 1e-6 to avoid zero divide
xe_h = lambda n_harm, undulator_N, dEpE: (2*np.pi*undulator_N*n_harm*dEpE+1e-6)

Qs_h = lambda n_harm, undulator_N, dEpE: Qsn_h(xe_h(n_harm, undulator_N, dEpE))
Qa_h = lambda n_harm, undulator_N, dEpE: Qan_h(xe_h(n_harm, undulator_N, dEpE))

Qs = Qs_h(n_harm, undulator_N, dEpE)
Qa = Qa_h(n_harm, undulator_N, dEpE)

# photon beam size (coherent limit)
sp_m    = np.sqrt(lambdas_m*2*undulator_L_m)/(2*np.pi)
spp_rad = np.sqrt(lambdas_m/(2*undulator_L_m))

# beam size with energy spread only
Sx2_m    = Qs*sp_m
Sxp2_rad = Qa*spp_rad
Sy2_m    = Qs*sp_m
Syp2_rad = Qa*spp_rad

# beam with finite ebeam size only
Sx3_m    = np.sqrt(sex_m**2+   sp_m**2)
Sxp3_rad = np.sqrt(sexp_rad**2+spp_rad**2)
Sy3_m    = np.sqrt(sey_m**2+   sp_m**2)
Syp3_rad = np.sqrt(seyp_rad**2+spp_rad**2)

# beam size with energy spread anf finite ebeam size only
Sx_m    = np.sqrt(sex_m**2+   (Qs*sp_m)**2)
Sxp_rad = np.sqrt(sexp_rad**2+(Qa*spp_rad)**2)
Sy_m    = np.sqrt(sey_m**2+   (Qs*sp_m)**2)
Syp_rad = np.sqrt(seyp_rad**2+(Qa*spp_rad)**2)


# coherent emittance limit
emittance0_m2_rad2 = sp_m*spp_rad*sp_m*spp_rad
# effective emittance
emittance_m2_rad2 =  Sx_m*Sy_m*Sxp_rad*Syp_rad

coh_frac = 1/(emittance_m2_rad2/(emittance0_m2_rad2))

### Brilliance & coherent flux

In [9]:
# norm_factor = (4/pi)^2*2
# brilliance_ph_s_mm2_mrad2_01bw = flux_ph_s_01bw ./  (emittance_m2_rad2*(2*pi)^2*1e12);
# semilogy(Es_eV, brilliance_ph_s_mm2_mrad2_01bw,'k')
# ylabel('Brilliance [ph/s/mm^2/mrad^2/0.1BW]')
# title('COSMIC ID brilliance (\Lambda_0=38mm, LP)')
# xlim([250 2500])
# set(gca,'xTick',[250:250:2500])

# % ylim([0.5e20 2e20])
# % set(gca,'yTick',[5e19 1e20 2e20 ])
# % ytickformat('%1.1f')

# ylim([2e19 2e20])
# set(gca,'yTick',[1e18 2e18 5e18 1e19 2e19 5e19 1e20 2e20])
# ytickformat('%1.0e')
# grid on


# %% coherent flux

# flux_h1_g1_coh_ph_s = flux1_ph_s_01bw.*eta1s./RP1*1000.*coh_frac;
